In [1]:
import matplotlib.pyplot as plt
from astropy import *
import astropy.units as u
from astropy.io import fits
import numpy as np
import glob as glob
import pandas as pd
from astropy.table import Table,vstack,hstack

import subprocess
import sys
import os
import logging
import numpy as np

from scipy.interpolate import interp1d
import time
from astropy.stats import sigma_clip
import scipy
from scipy.interpolate import interp1d
from scipy import interpolate

In [2]:
# ! ls

In [3]:
path = '/uufs/chpc.utah.edu/common/home/astro/zasowski/sinha/fits/'


galah = Table.read(path + 'galah_dr4_allstar.fits')
unique_ids,indices,counts = np.unique(galah['sobject_id'],return_index=True,return_counts=True)
galah = galah[indices]

wide_binaries = Table.read(path + 'all_columns_catalog.fits')

ipl4 = Table.read('/uufs/chpc.utah.edu/common/home/sdss50/ipl-4/spectro/astra/summary/astraAllStarASPCAP-0.8.fits.gz',hdu=2)
unique_sdss_ids = np.unique(ipl4['sdss_id'])
ipl4.sort('snr',reverse=True)
unique_ids,indices,counts = np.unique(ipl4['sdss_id'],return_index=True,return_counts=True)
ipl4 = ipl4[indices]



In [4]:
msms_wide_binaries = wide_binaries[wide_binaries['binary_type']=='MSMS']

truth1 = np.isin(msms_wide_binaries['source_id1'],ipl4['gaia_dr3_source_id'])
truth2 = np.isin(msms_wide_binaries['source_id2'],ipl4['gaia_dr3_source_id'])

truth = truth1 & truth2

ipl4_wb_candidates = msms_wide_binaries[truth]

In [5]:
truth1 = np.isin(msms_wide_binaries['source_id1'],galah['gaiadr3_source_id'])
truth2 = np.isin(msms_wide_binaries['source_id2'],galah['gaiadr3_source_id'])

truth = truth1 & truth2

galah_wb_candidates = msms_wide_binaries[truth]

In [6]:
galah_wb_candidates['pair'] = np.arange(1, len(galah_wb_candidates) + 1, 1)
galah['pair'] = -9999

In [11]:
truth1 = np.isin(galah['gaiadr3_source_id'],galah_wb_candidates['source_id1'])
truth2 = np.isin(galah['gaiadr3_source_id'],galah_wb_candidates['source_id2']) 
truth = truth1 | truth2

galah_wbs = galah[truth]

for i in galah_wbs:
    row = galah_wb_candidates[(galah_wb_candidates['source_id1']==i['gaiadr3_source_id']) | (galah_wb_candidates['source_id2']==i['gaiadr3_source_id'])]
    i['pair'] = row['pair']

In [12]:
galah_wbs['companion_type'] = 'idk'

temp = []
for i in np.unique(galah_wbs['pair']):
    members = galah_wbs[galah_wbs['pair']==i]
    members.sort('teff',reverse=True)
    members['companion_type'] = ['primary','secondary']
    temp.append(members)
galah_wbs = vstack(temp)

In [13]:
good_galah_wbs = galah_wbs[galah_wbs['snr_px_ccd3'] > 50]
unique_ids,indices,counts = np.unique(good_galah_wbs['pair'],return_index=True,return_counts=True)

good_galah_wbs = good_galah_wbs[np.where(counts>1)]

In [14]:
n_wbs = 0
good_pairs = []
for i in np.unique(good_galah_wbs['pair']):
    members = good_galah_wbs[good_galah_wbs['pair']==i]
    sec = members[members['companion_type']=='secondary']
    pri = members[members['companion_type']=='primary']

    if (sec['teff'] < 4200) and (pri['teff'] > 4200) and (pri['teff'] < 6000) and (sec['snr_px_ccd3'] > 50) and (pri['snr_px_ccd3'] > 50):
        n_wbs +=1
        good_pairs.append(i)

/scratch/local/u1363702/6688992/ipykernel_1094358/2515849036.py:8: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if (sec['teff'] < 4200) and (pri['teff'] > 4200) and (pri['teff'] < 6000) and (sec['snr_px_ccd3'] > 50) and (pri['snr_px_ccd3'] > 50):


In [15]:
n_wbs

2

In [16]:
usable_wbs = good_galah_wbs[np.isin(good_galah_wbs['pair'], good_pairs)]
usable_wbs.write('m_dwarf_g_dwarf_wide_binaries.fits',overwrite=True)

In [17]:
# get all usable m-dwarfs

In [18]:
truth = (galah['teff'] < 4200) & (galah['logg'] > 3.5) & (galah['snr_px_ccd3'] > 100)
galah_mdwarfs = galah[truth]
# mdwarfs.write('galah_m_dwarf_candidates.fits',overwrite=True)

In [20]:
# crossmatch apogee-galah

In [22]:
truth = (ipl4['raw_teff'] < 4200) & (ipl4['raw_logg'] > 3.5) & (ipl4['snr'] > 100)
ipl4_mdwarfs = ipl4[truth]
# ipl4_mdwarfs.write('ipl4_m_dwarf_candidates.fits',overwrite=True)

In [23]:
# galah_ipl4_mdwarfs = 
truth = np.isin(ipl4_mdwarfs['gaia_dr3_source_id'],galah_mdwarfs['gaiadr3_source_id'])
ipl4_crossmatches = ipl4_mdwarfs[truth]

truth = np.isin(galah_mdwarfs['gaiadr3_source_id'],ipl4_mdwarfs['gaia_dr3_source_id'])
galah_crossmatches = galah_mdwarfs[truth]

ipl4_crossmatches.sort('gaia_dr3_source_id')
galah_crossmatches.sort('gaiadr3_source_id')

galah_crossmatches['sdss_id'] = ipl4_crossmatches['sdss_id']
galah_crossmatches['sdss_teff'] = ipl4_crossmatches['raw_teff']
galah_crossmatches['sdss_logg'] = ipl4_crossmatches['raw_logg']
galah_crossmatches['sdss_m_h'] = ipl4_crossmatches['raw_fe_h']


In [25]:
galah_crossmatches.write('crossmatched_mdwarfs.fits',overwrite=True)